In [47]:
import dataclasses
import typing
import enum
from typing import Any
test_values = [None]*100000 + list(range(100000))

In [48]:
# option 1: baseline - check flag before using
@dataclasses.dataclass
class CheckDataType:
    a: int
    def a_is_missing(self) -> bool:
        return self.a is None
    
    def a_is_not_even(self) -> bool:
        return self.a % 2 == 0

def average_values_check(objs: typing.List[CheckDataType]):
    values = list()
    for obj in objs:
        if not obj.a_is_missing():
            if obj.a_is_not_even():
                values.append(obj.a)
            else:
                values.append(0)
    return sum(values)/len(values)

check_objs = [CheckDataType(v) for v in test_values]
%timeit average_values_check(check_objs)

34.1 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
# option 2: use exceptions

class ValueIsMissing(BaseException):
    pass

class ValueIsNotEven(BaseException):
    pass

@dataclasses.dataclass
class ExcDataType:
    a: int
    def access_a(self) -> ReturnStatus:
        if self.a is None:
            raise ValueIsMissing('The data for a is missing.')
        elif self.a % 2 == 0:
            return self.a
        else:
            raise ValueIsNotEven('The data for a is missing.')

def average_values_exc(objs: typing.List[ExcDataType]):
    values = list()
    for obj in objs:
        try:
            values.append(obj.access_a())
        except ValueIsMissing:
            pass
        except ValueIsNotEven:
            values.append(0)
    return sum(values)/len(values)

exc_objs = [ExcDataType(v) for v in test_values]
%timeit average_values_exc(exc_objs)

65.6 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
# option 3b: use result object
class Status(enum.Enum):
    Ok = enum.auto()
    Err = enum.auto()

class ErrorTypeNotHandled(BaseException):
    pass
    
class ReturnStatus:
    def handle(self, 
            error_handlers: typing.Dict[enum.Enum, typing.Callable[[Any],Any]],
            okay_func: typing.Callable[[Any],Any] = lambda x: x,
        ) -> typing.Any:
        if self.status is Status.Ok:
            return okayfunc(self.data)
        else:
            try:
                return error_handlers[self.error](**self.error_kwargs)
            except KeyError as e:
                raise ErrorTypeNotHandled(f'Error type {self.error} was not handled') from e

@dataclasses.dataclass
class Ok(ReturnStatus):
    data: typing.Any
    status: Status = Status.Ok

@dataclasses.dataclass
class Err(ReturnStatus):
    error: BaseException
    error_kwargs: typing.Dict = dataclasses.field(default_factory=dict)
    status: Status = Status.Err

class ErrorType(enum.Enum):
    MISSING = enum.auto()
    NOT_EVEN = enum.auto()
    
@dataclasses.dataclass
class StatusDataType:
    a: int
    def access_a(self) -> ReturnStatus:
        if self.a is None:
            return Err(ErrorType.MISSING)
        elif self.a % 2 == 0:
            return Ok(self.a)
        else:
            return Err(ErrorType.NOT_EVEN)

def average_values_status(objs: typing.List[StatusDataType]):
    values = list()
    for obj in objs:
        result = obj.access_a()
        if result.status is Status.Ok:
            values.append(result.data)
        elif result.error is ErrorType.NOT_EVEN:
            values.append(0)
    return sum(values)/len(values)

status_objs = [StatusDataType(v) for v in test_values]
%timeit average_values_status(status_objs)


def raise_(exception: BaseException):
    raise exception

b = MyType(1).access_a().handle({
    ErrorType.MISSING: lambda: raise_(ValueError()), 
    ErrorType.NOT_EVEN: lambda: None,
})
#test_values = [None]*10 + list(range(100))
#objs = [OtherDataType(v) for v in test_values]
#%timeit [get_value(o) for o in objs]

155 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
class ReturnStatus:
    @property
    def is_ok(self) -> bool:
        return isinstance(self, Ok)

@dataclasses.dataclass
class Ok(ReturnStatus):
    data: typing.Any

@dataclasses.dataclass
class Err(ReturnStatus):
    error: ErrorType

class ErrorType(enum.Enum):
    MISSING = enum.auto()
    NOT_EVEN = enum.auto()
    
@dataclasses.dataclass
class Status2DataType:
    a: int
    def access_a(self) -> ReturnStatus:
        if self.a is None:
            return Err(ErrorType.MISSING)
        elif self.a % 2 == 0:
            return Ok(self.a)
        else:
            return Err(ErrorType.NOT_EVEN)

def average_values_status(objs: typing.List[StatusDataType]):
    values = list()
    for obj in objs:
        result = obj.access_a()
        if result.is_ok:
            values.append(result.data)
        elif result.error is ErrorType.NOT_EVEN:
            values.append(0)
    return sum(values)/len(values)

status_objs = [StatusDataType(v) for v in test_values]
%timeit average_values_status(status_objs)


def raise_(exception: BaseException):
    raise exception

b = MyType(1).access_a().handle({
    ErrorType.MISSING: lambda: raise_(ValueError()), 
    ErrorType.NOT_EVEN: lambda: None,
})
#test_values = [None]*10 + list(range(100))
#objs = [OtherDataType(v) for v in test_values]
#%timeit [get_value(o) for o in objs]

127 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


AttributeError: 'Err' object has no attribute 'handle'

In [18]:
import dataclasses
import typing
import enum
from typing import Any

class Status(enum.Enum):
    Ok = enum.auto()
    Err = enum.auto()

class ReturnStatus:
    status = None
    def is_okay(self) -> bool:
        return self.status is Status.Ok
    def handle(self, 
            err_funcs: typing.Dict[enum.Enum, typing.Callable[[Any],Any]],
            okay_func: typing.Callable[[Any],Any] = lambda x: x,
        ) -> typing.Any:
        if self.is_okay():
            return okayfunc(self.data)
        else:
            return err_funcs[self.error_type](*self.args, **self.kwargs)

@dataclasses.dataclass
class Ok(ReturnStatus):
    data: typing.Any
    status: Status = Status.Ok

class Err(ReturnStatus):
    def __init__(self,
        error_type: typing.Any,
        *args,
        status: Status = Status.Err,
        **kwargs,
    ):
        self.error_type = error_type
        self.status = status
        self.args = args
        self.kwargs = kwargs

    
class ErrorType(enum.Enum):
    MISSING = enum.auto()
    NOT_EVEN = enum.auto()
    
@dataclasses.dataclass
class MyType:
    a: int
    def access_a(self) -> ReturnStatus:
        if self.a is None:
            return Err(ErrorType.MISSING)
        elif self.a % 2 == 0:
            return Ok(self.a)
        else:
            return Err(ErrorType.NOT_EVEN)

def raise_(exception: BaseException):
    raise exception

fme = 54
a = MyType(1).access_a().handle({
    ErrorType.MISSING: lambda: raise_(ValueError()), 
    ErrorType.NOT_EVEN: lambda: None,
})
a

True

In [ ]:
class ValueIsNotEven(BaseException):
    pass
class ValueIsMissing(BaseException):
    pass

@dataclasses.dataclass
class OtherDataType:
    a: int
    def access_a(self) -> ReturnStatus:
        if self.a is None:
            raise ValueIsMissing('The data for a is missing.')
        elif self.a % 2 == 0:
            return self.a
        else:
            raise ValueIsNotEven('The data for a is missing.')

            
def get_value(obj: OtherDataType):
    try:
        return obj.access_a()
    except ValueIsMissing as e:
        return None

test_values = [None]*10 + list(range(100))
objs = [OtherDataType(v) for v in test_values]
%timeit [get_value(o) for o in objs]